<a href="https://colab.research.google.com/github/Alecia113/NLP-Emotional-analysis/blob/main/AE4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  

import pickle #必要的
training_data = pickle.load(open("training_data.pkl","rb")) #必要的
testing_data = pickle.load(open("testing_data.pkl","rb")) #必要的

print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data))) #8000
print("Size of testing dataset: {0}".format(len(testing_data))) #2000
print("------------------------------------")

print("------------------------------------")
print("Sample Data")
print("LABEL: {0} / SENTENCE: {1}".format(training_data[0][0], training_data[0][1]))
print("------------------------------------")
#LABEL: neg / SENTENCE: hopeless for tmr :(

------------------------------------
Size of training dataset: 8000
Size of testing dataset: 2000
------------------------------------
------------------------------------
Sample Data
LABEL: neg / SENTENCE: hopeless for tmr :(
------------------------------------


In [27]:
import torch
#You can enable GPU here (cuda); or just CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#分集合

train_data = [training_data[i][1] for i in range(len(training_data))]
train_label = [training_data[i][0] for i in range(len(training_data))]

test_data = [testing_data[i][1] for i in range(len(testing_data))]  
test_label = [testing_data[i][0] for i in range(len(testing_data))] #neg pos

#做预处理 变小写
import pprint
text_train = [s.lower() for s in train_data]
text_test = [s.lower() for s in test_data]

# 分词
import pprint
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()


train_t=[]  #完全每句话的分词的集合。train_t[0][0]每个单词
for s in text_train:
  text_train = tknzr.tokenize(s)                         #每句话分别作了分词 text_train[0]第一句话的分词
  train_t.append(text_train)  
        #:(    :-(


test_t=[]     #2000
for w in text_test:
  text_test = tknzr.tokenize(w)
  test_t.append(text_test)

#删
#theprincesszooz but i see what youre going at   
#yes  subjective pain may not be real
#  but that does not make it less painful

import re
def remove(x):
  t = []
  for i in range(len(x)):
    t_sub = []        #是直接用空列表代替了
    for j in range(len(x[i])):
      if len(x[i][j])==0:
        continue
      if x[i][j] == " ":
        continue 
      else:
        x[i][j] = re.sub(r'\d','',x[i][j])                  #删除0-9
        if len(x[i][j]) == 1 or len(x[i][j]) > 3:           #and + or 问题。
          x[i][j] = re.sub(r'[^\w\s+]','', x[i][j])
        if len(x[i][j]) == 0:
          continue 
        else:
          t_sub.append(x[i][j]) 
    t.append(t_sub) 

  return t

text_train = remove(train_t)
text_test = remove(test_t)

##停用词  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as sw
stop_words = sw.words()
sww = sw.words()

text_train_ns=[]    #8000
for tokens in text_train:
    filtered_sentence1 = [w for w in tokens if not w in stop_words]
    text_train_ns.append(filtered_sentence1)


text_test_ns=[]   #2000
for tokens in text_test:
  filtered_sentence2 = [w for w in tokens if not w in stop_words]
  text_test_ns.append(filtered_sentence2)

##找词根  没什么用，全是00
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

text_train_le = []
for tokens in text_train_ns:
  lemma_sentence1 = [lemmatizer.lemmatize(w) for w in tokens ]
  text_train_le.append(lemma_sentence1)

print('\n')
text_test_le = []
for tokens in text_test_ns:
  lemma_sentence2 = [lemmatizer.lemmatize(w) for w in tokens]
  text_test_le.append(lemma_sentence2)




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[['hopeless', 'tmr', ':('], ['everyth', 'kid', 'section', 'ikea', 'cute', 'shame', "i'm", 'nearli', 'month', ':('], ['hegelbon', 'heart', 'slide', 'wast', 'basket', ':('], ['ketchburn', 'hate', 'japanes', 'call', 'bani', ':(', ':('], ['dang', 'start', 'next', 'week', 'work', ':(']]
[['andreamarysmith', 'help', '...', 'stop', 'cri', ':('], ['realyys_', 'otl', 'nevermind', ':(', 'least', 'got', 'jeon'], ['soon', 'tweet', 'plant', 'claw', 'thigh', 'traction', 'zoom', 'away', ':('], ['luketothestar', 'damnit', ':('], ['klm', 'use', 'pri', 'pv', '...', 'wish', 'could', 'reliv', 'day', 'becom', 'nyc', 'pv', 'buy', 'way', 'commun', 'nyc', 'usa', 'klm', ':(']]


In [ ]:
#text_test_le      text_train_le

#WORD embedding

In [28]:
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences= text_train_le + text_test_le, size=100, window=5, min_count=2, workers=2, sg=0)


In [13]:
#否定词"ISO-8859-1"==UTF-8
from google.colab import drive
drive.mount('/content/drive') #必须要挂载
not_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding="ISO-8859-1").readlines()]
#处理文件呢。
j = -1
for i in not_words:
  j +=1
  if i == '2-faced':
    break
neg = not_words[j:]

#处理文件呢。
sure_words = [w.strip() for w in open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt', 'r', encoding="ISO-8859-1").readlines()]

j = -1
for i in sure_words:
  j +=1
  if i == 'a+':
    break

pos = sure_words[j:]
print(len(pos)) #2006
print(pos[0]) #a+
print(pos[:10])

#neg pos 词表
#要对他们做stem因为训练集做了stem


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
neg_le = []
for tokens in neg:
  lemma_sentence2 = [lemmatizer.lemmatize(tokens)]
  neg_le.append(lemma_sentence2)
print(neg_le[:5])

pos_le = []
for tokens in pos:
  lemma_sentence2 = [lemmatizer.lemmatize(tokens)]
  pos_le.append(lemma_sentence2)
print(pos_le[:5])

[['2-faced'], ['2-faces'], ['abnormal'], ['abolish'], ['abominable']]
[['a+'], ['abound'], ['abounds'], ['abundance'], ['abundant']]


In [26]:
#只需要先让他们对应起来。您要检查每个单词是在正还是负词典中not_exist（0），negative（1）和positive（2）
#一维句子，二维单词
#是要吧一句话变成【012组成的形式】
#现在就是一一对应的形式了。一句话;这个是train_stem【 as,ada,daff】 match_embedding【120】同一个索引在两个列表中 
train_match= []
for i in range(len(text_train_le)):
  match = []
  for j in range(len(text_train_le[i])):
    if text_train_le[i][j] in neg_le[0]:
      match.append(1)
    elif text_train_le[i][j] in pos_le[0]:
      match.append(2)
    else:
      match.append(0)
  train_match.append(match)
print(train_match)
print(text_train_le[:5])

[[0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0], [0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
test_match= []
for i in range(len(test_stem)):
  match = []
  for j in range(len(test_stem[i])):
    if test_stem[i][j] in neg_stem[0]:
      match.append(1)
    elif test_stem[i][j] in pos_stem[0]:
      match.append(2)
    else:
      match.append(0)
  test_match.append(match)
print(test_match[:5])
print(test_stem[:5])

In [ ]:
#bug1
#只需要先让他们对应起来。您要检查每个单词是在正还是负词典中not_exist（0），negative（1）和positive（2）
#一维句子，二维单词
#是要吧一句话变成【012组成的形式】
#现在就是一一对应的形式了。一句话;这个是train_stem【 as,ada,daff】 match_embedding【120】同一个索引在两个列表中 
'''
match_embedding = []
for i in range(len(train_stem)):
  match = []
  for j in range(len(train_stem[i])):
    if train_stem[i][j] == [neg_stem[0][n] for n in range(len(neg_stem))]:
      match.append(1)
    elif train_stem[i][j] == [pos_stem[0][n] for n in range(len(pos_stem))]:
      match.append(2)
    else:
      match.append(0)
  match_embedding.append(match)
print(match_embedding)
'''

In [ ]:
#想实现的是： 让train_stem[0].vec + match_embedding[0] = input 
#wv_cbow_model["neg[787]"] 
'''
>>> torch.cat((x, x, x), 1)
tensor([[ 0.6580, -1.0969, -0.4614,  0.6580, -1.0969, -0.4614,  0.6580,
         -1.0969, -0.4614],
        [-0.1034, -0.5790,  0.1497, -0.1034, -0.5790,  0.1497, -0.1034,
         -0.5790,  0.1497]])
'''
#torch.cat((wv_cbow_model["train_stem[i][j]"] , match_embedding[i][j]),1) 

In [ ]:
wv_cbow_model["hopeless"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.00732632, -0.00777728,  0.00699988, -0.0020741 ,  0.00235598,
        0.00311979,  0.00641038, -0.00537038,  0.00875941,  0.00242392,
        0.00582156, -0.00991632, -0.00481745, -0.01309411, -0.00210777,
       -0.01110752,  0.00417562,  0.00844241, -0.01010149, -0.00215971,
        0.01101657,  0.00244154, -0.01167845,  0.01250843, -0.0070031 ,
        0.01267646,  0.00909506, -0.00955854,  0.02015485,  0.00299432,
        0.00354427, -0.00828324,  0.00039551, -0.00329358, -0.0127624 ,
        0.01075943,  0.00910194,  0.00368676, -0.02225151,  0.00173635,
       -0.00158221, -0.01030921,  0.00840114,  0.00799288, -0.00369217,
        0.00203344,  0.00014375, -0.00500246,  0.0003207 ,  0.00964787,
       -0.00784805,  0.000375  , -0.00447435, -0.0106426 , -0.00611883,
        0.01357756,  0.01787031,  0.00441383, -0.01512035,  0.01083756,
       -0.01627806, -0.01369711, -0.00579861, -0.00793204,  0.01050298,
        0.00238665, -0.00711585, -0.00740496,  0.0099582 , -0.00

In [ ]:
word_vec = torch.tensor(wv_cbow_model["hope"],dtype=torch.long)
lexicon = torch.tensor(0,dtype=torch.long)
input = torch.cat((word_vec, lexicon),1) 

In [ ]:
'''
for i in range(len(train_stem)):
  for j in range(len(train_stem[i])):
    if train_match[i][j] != 0:
      word_vec = torch.tensor(wv_cbow_model[train_stem[i][j]],dtype=torch.long)
      lexicon = torch.tensor(train_match[i][j],dtype=torch.long)
      input = torch.cat((word_vec, lexicon),1) 
    print(input)
  # k = torch.tensor(wv_cbow_model["train_stem[i][j]"],dtype=torch.long) 
  #   #wv_cbow_model["train_stem[i][j]"]  + match_embedding[i][j] = input
  #只有一个想法为啥会跑崩溃
'''

In [ ]:
#然后，您要检查每个单词是在正还是负词典中
#去掉复数 stemmer 词根不需要这部； 如果用就需要在neg和pos需要stemmer操作
'''
stemmer = PorterStemmer()
#text_train_le = [text_train_le[i] for i in range(len(text_train_le))]
#singles = [stemmer.stem(plural) for plural in text_train_le[i] for i in range(len(text_train_le))]
for i in range(len(neg)):
  for plural in neg:
    singles3 = stemmer.stem(plural)
print(singles3[:5])

for i in range(len(neg)):
  for plural in neg:
    singles4 = stemmer.stem(plural)
print(singles3[:5])
'''

In [ ]:
stop

NameError: ignored

In [ ]:
#【不要】学习加载文件
'''
with open('/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt', 'r', encoding='utf-8') as f: f.write('This method is prior')
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/negative-words.txt
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English/positive-words.txt
#/content/drive/MyDrive/NLP/A1/E/opinion-lexicon-English

'''
'''
#加载数据，要知道文件夹在哪里，导入数据集
#显示数据集里有什么
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/train/images_training.h5", "r") as f:
  data_train = np.copy(f["datatrain"])
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/train/labels_training.h5", "r") as f:
  label_train = np.copy(f["labeltrain"])
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/test/images_testing.h5", "r") as f:
  data_test = np.copy(f["datatest"])  
with h5py.File("/content/drive/My Drive/5318a1/Assignment/Input/test/labels_testing_2000.h5", "r") as f:
  label_test = np.copy(f["labeltest"])
  '''

#train

In [ ]:
wv_cbow_model.save("cbow.model")
#训练 ； 存 取；训练。
cbow = Word2Vec.load("./cbow.model")
cbow.train(sentences= train_stem + test_stem ,total_examples=wv_cbow_model.corpus_count, epochs=cbow.iter)
#(235832, 368710)
#https://blog.csdn.net/yibo492387/article/details/81908249?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161909968516780261919053%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161909968516780261919053&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-1-81908249.first_rank_v2_pc_rank_v29&utm_term=Word2vec.train
#model.train([["hopeless", "weekend"]]), total_examples=1, epochs=1)  # #基于新语料更新词向量表示
#https://blog.csdn.net/baidu_26550817/article/details/48653889?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780262535664%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161910098016780262535664&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~baidu_landing_v2~default-1-48653889.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数
#【上面是解释训练的这些参数是什么】
#https://blog.csdn.net/qq_19707521/article/details/79168239?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780262535664%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=161910098016780262535664&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~baidu_landing_v2~default-2-79168239.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数
#https://blog.csdn.net/UZDW_/article/details/114374822?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161910098016780261927889%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161910098016780261927889&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-7-114374822.first_rank_v2_pc_rank_v29&utm_term=word2vec.train参数


In [ ]:
vector = cbow.wv['check']
vector

In [ ]:
#给出的训练
'''
model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
model.save("word2vec.model")
model2 = Word2Vec.load("./word2vec.model")
model2.train(common_texts,total_examples=model.corpus_count, epochs=model2.iter)
 '''
'''
from gensim.models import Word2Vec as wtv
word_model = wtv.load("C:/model/word2vec_model")  # 加载模型词向量参数
word_model.train([["我","爱","你","们"]], total_examples=1, epochs=10)  #基于新语料更新词向量表示
【不断加载模型】

 这种方法可以对现有word2vec的词向量进行更新，在实操的时候，train函数中的训练数据如果出现新的词，并不会更新到现有的词向量矩阵中。

举个例子，已经训练好的word2vec词表有[“我”,“爱”,“你”]三个词，给定一个新语料[“我”,“喜”,“欢”,“你”]，新语料只会调整原有词中的“我”“你”的词向量表示，而不会将“喜”“欢”添加到词库中。
#https://blog.csdn.net/qq_37388085/article/details/108289649?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522161909860016780271575634%2522%252C%2522scm%2522%253A%252220140713.130102334.pc%255Fall.%2522%257D&request_id=161909860016780271575634&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~first_rank_v2~rank_v29-11-108289649.first_rank_v2_pc_rank_v29&utm_term=如何直接训练Word2Vec
'''

**2.2 Lexicon Embedding**

In [ ]:
'''
#读取否定词文件
not_word_file = open('否定词.txt','r+',encoding='utf-8')
not_word_list = not_word_file.readlines()
degree_dict = defaultdict()
'''

In [ ]:
word_set = set() 
for sent in text_train_le:#获取单词列表的输出样本text_train_le    text_test_le
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index) #word_index
#这里需要的是word_list

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
#应该是seq2seq
seq_length = 100   #回头要弄 [得弄个for循环看看最长句子要多少]

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 
print(train_pad_encoded[:2])

In [ ]:
# Get the Embedding lookup table
import numpy as np
emb_dim = wv_cbow_model.vector_size

emb_table = []
for i, word in enumerate(word_list):
    if word in wv_cbow_model:
        emb_table.append(wv_cbow_model[word])
    else:
        emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #目前是个array

In [ ]:
print(emb_table.shape) #(16283, 100)

In [ ]:
lookup_tensor = torch.tensor(emb_table,dtype=torch.long) #是不是全是0啊？？？？把array变tensor

In [ ]:
lookup_tensor.shape #torch.Size([16283, 100])

In [ ]:
#lab3 所以训练要用nn.model
voc_size = len(word_list)
learning_rate = 0.1
batch_size = 16
embedding_size = 2
no_of_epochs = 5000

test

In [ ]:
emb_table

In [ ]:
lookup_tensor = torch.tensor(emb_table,dtype=torch.long) #是不是全是0啊？？？？

In [ ]:
emb_table #16283,100  #这个是个100
emb_table[1] #
lookup_tensor[2]

In [ ]:
wv_cbow_model.wv['hopeless']

**Get Word List**

In [ ]:
word_set = set() 
for sent in text_train_le:#获取单词列表的输出样本text_train_le    text_test_le
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index) #word_index

**padding and encoding [Please complete this section]**

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16   #回头要弄 [得弄个for循环看看最长句子要多少]

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

In [ ]:
train_pad_encoded[0]

2)Lexicon Embedding (2 marks) 012 还要处理OVV

#TEST

In [ ]:
len(train_pad_encoded) # [5549, 7921, 2087, 1222, 38, 38, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72],

In [ ]:
wv_cbow_model.vector_size

#**Model**

In [ ]:
'''
1 要先准备数据； 做分词预处理的行为。
差的是构建模型。
2 要构建训练模型： wword（………………………………）
只是创建完，回头数据还是要改。但明确了哪里是哪里很重要。
'''

# Initialize and train a word2vec model with the following parameters:
# sentence: iterable of iterables, i.e. the list of lists of tokens from our data
# size: dimensionality of the word vectors
# window: window size
# min_count: ignores all words with total frequency lower than the specified count value
# workers: Use specified number of worker threads to train the model (=faster training with multicore machines)
# sg: training algorithm, 0 for CBOW, 1 for skip-gram
from gensim.models import Word2Vec
wv_cbow_model = Word2Vec(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(wv_cbow_model.most_similar(":("))
wv_cbow_model = Word2Vec(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(wv_cbow_model.wv.most_similar(":("))
print(wv_cbow_model.wv.most_similar(":(") == wv_cbow_model.most_similar(":("))


In [ ]:
#word2vec skip gram
# Now we switch to a Skip Gram model by setting parameter sg=1
wv_sg_model = Word2Vec(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(wv_sg_model.most_similar(":("))
wv_sg_model = Word2Vec(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=1)

print(wv_sg_model.wv.most_similar(":("))
print(wv_sg_model.wv.most_similar(":(") == wv_sg_model.most_similar(":("))


In [ ]:
#FastText -skip gram
from gensim.models import FastText
# Now we initialize and train FastText with Skip Gram architecture (sg=1)
ft_sg_model = FastText(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(ft_sg_model.most_similar(":("))
ft_sg_model = FastText(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=1)
print(ft_sg_model.wv.most_similar(":("))

In [ ]:
#FastText -cbow
# Now we initialize and train FastText with CBOW architecture (sg=0)
ft_cbow_model = FastText(sentences=text_train_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(ft_cbow_model.most_similar(":("))
ft_cbow_model = FastText(sentences=text_test_le, size=100, window=5, min_count=5, workers=2, sg=0)
print(ft_cbow_model.most_similar(":("))

#Lexicon embeddings 不是这个是要输入句子。把句子加上word+embedding

**Label encodding[标签编码]**

In [ ]:
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
#To normalise the data (so that the variance of the feature data is 1 and the mean is 0), the mean μ and variance σ^2 of the feature data are calculated. fit is not used because the training set (fit) has already found the mean and variance. (The transformation rule is found), and this rule is used directly.
label_train_encoded = lEnc.fit_transform(train_label)
label_test_encoded = lEnc.transform(test_label)

n_class = len(unique_labels)
print(unique_labels)
print(lEnc.transform(unique_labels))
#not_exist(0), negative(1), and positive(2)


**get word list**

In [ ]:
word_index['[PAD]']   #72 [] 不是（）
word_index['[UNKNOWN]']# 73

**padding and encoding**

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

*[可选]您需要说明为什么您想使用超过一维的嵌入。
*[Optional] You are required to describe why you would like to use more than one-dimensional embedding.*

In [ ]:
wv_cbow_model.vector_size

In [ ]:
word_set = set() 
for sent in text_train_le:
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index)

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】

import numpy as np
emb_dim = wv_cbow_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in wv_cbow_model:
    emb_table.append(wv_cbow_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #16283

In [ ]:
emb_table.shape#(16283, 100)

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】

import numpy as np
emb_dim = word_emb_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in word_emb_model:
    emb_table.append(word_emb_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)  #16283

# Download Embeddings:

In [ ]:
#https://github.com/RaRe-Technologies/gensim-data
#下载 预训练 嵌入 pre_trained embedding
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25") # corpus = api.load('text8')
#还有只下载路径不打开的。
'''
import gensim.downloader as api
print(api.load("20-newsgroups", return_path=True))  # output: /home/user/gensim-data/20-newsgroups/20-newsgroups.gz
print(api.load("glove-twitter-25", return_path=True))  # output: /home/user/gensim-data/glove-twitter-25/glove-twitter-25.gz
'''


In [ ]:
word_emb_model

In [ ]:
word_emb_model.most_similar("apple") #和它相似单词的词向量。

In [ ]:
#例如，load 一个语料库，+ 去训练word2vec model
tran_model = Word2Vec(word_emb_model) #NameError: name 'Word2Vec' is not defined
tran_model.most_similar("car")

#build word embeddings model

**Label encodding[标签编码]**

In [ ]:
import numpy as np

from sklearn.preprocessing import LabelEncoder

unique_labels = np.unique(train_label)

lEnc = LabelEncoder()
# Please encode the labels (Do NOT add new lines of code in this section)
# Hint: Try to understand the difference between fit_transform and transform
#To normalise the data (so that the variance of the feature data is 1 and the mean is 0), the mean μ and variance σ^2 of the feature data are calculated. fit is not used because the training set (fit) has already found the mean and variance. (The transformation rule is found), and this rule is used directly.
label_train_encoded = lEnc.fit_transform(train_label)
label_test_encoded = lEnc.transform(test_label)

n_class = len(unique_labels)
print(unique_labels)
print(lEnc.transform(unique_labels))


**get word list**

In [ ]:
word_set = set() 
for sent in text_train_le:
    for word in sent:
        word_set.add(word)
word_set.add('[PAD]')
word_set.add('[UNKNOWN]')

word_list = list(word_set) 
word_list.sort()
print(word_list)

word_index = {}
ind = 0
for word in word_list:
    word_index[word] = ind
    ind += 1
print(word_index)

**padding and encoding**

In [ ]:
# The sequence length is pre-defined, you can't change this value for this exercise
seq_length = 16

# Please Complete this function
# Hint: You should pay attention to: (1) if the sentence length > seq_length (2) if the word not in word_index dictionary
def encode_and_add_padding(sentences, seq_length, word_index):
    sent_encoded = []
    for sent in sentences: 
      if word not in sent:
        temp_encoded = [word_index.get(i,word_index['[UNKNOWN]']) for i in sent]       
      else:
        temp_encoded = [word_index[word] for word in sent]

      if len(temp_encoded) < seq_length:
        temp_encoded += [word_index['[PAD]']] * (seq_length - len(temp_encoded))  #30
      elif len(temp_encoded) > seq_length:
        temp_encoded = temp_encoded[:16]
      sent_encoded.append(temp_encoded)

    return sent_encoded

train_pad_encoded = encode_and_add_padding(text_train_le, seq_length, word_index )
test_pad_encoded = encode_and_add_padding(text_test_le, seq_length, word_index )  #100 16 

In [ ]:
train_pad_encoded #[3259, 7339, 1161, 11990, 15077, 40, 72, 72, 72, 72, 72, 72, 72, 72, 72, 72]

In [ ]:
#下载embedding
'''
import gensim.downloader as api
word_emb_model = api.load("xxx") # Download an embedding other than glove-twitter-25
'''

In [ ]:
#创建嵌入查询表  embedding lookup table 【是传说中的W吗？】 【为什么要嵌入这个词表】
import numpy as np
emb_dim = word_emb_model.vector_size    #要训练的数据集的向量大小
emb_table = []
for i, word in enumerate(word_list):  #['[PAD]', '[UNKNOWN]', 'do', 'hate', 'i', 'it', 'like', 'love', 'not', 'that']
  if word in word_emb_model:
    emb_table.append(word_emb_model[word])
  else:
    emb_table.append([0]*emb_dim)
emb_table = np.array(emb_table)
print(emb_table)

#WordEmbedding

In [ ]:
id = '1B47OiEiG2Lo1jUY6hy_zMmHBxfKQuJ8-'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('ted_en-20160408.xml')  



In [ ]:
# 下载Twitter data
import nltk
nltk.download('twitter_samples')
from nltk.corpus import twitter_samples

In [ ]:
#look like
twitter_samples.fileids()
print("Negative LENGTH : " +  str(len(twitter_samples.strings('negative_tweets.json'))) )
print("Positive LENGTH : " +  str(len(twitter_samples.strings('positive_tweets.json'))) )
twitter_samples.strings('positive_tweets.json')[:2]
twitter_samples.strings('negative_tweets.json')[:2]

In [ ]:
#putting labels
label = "neg"
neg_dataset = [(label, instance) for instance in twitter_samples.strings('negative_tweets.json')]

label = "pos"
pos_dataset = [(label, instance) for instance in twitter_samples.strings('positive_tweets.json')]

print(neg_dataset[:2])
print(pos_dataset[:2])

In [ ]:
#splitting dataset 分隔

training_data = neg_dataset[:4000] + pos_dataset[:4000]
testing_data = neg_dataset[4000:] + pos_dataset[4000:]


####### relatively small dataset, can be used at development phase #######
dev_training_data = neg_dataset[:100] + pos_dataset[:100]
dev_testing_data = neg_dataset[100:200] + pos_dataset[100:200]


print("------------------------------------")
print("Size of training dataset: {0}".format(len(training_data)))
print("Size of testing dataset: {0}".format(len(testing_data)))
print("------------------------------------")
print("Size of dev training dataset: {0}".format(len(dev_training_data)))
print("Size of dev testing dataset: {0}".format(len(dev_testing_data)))
print("------------------------------------")

In [ ]:
#saving dataset into pickle files
import pickle
# write to cPickle
pickle.dump(training_data, open( "training_data.pkl", "wb" ) )
pickle.dump(testing_data, open( "testing_data.pkl", "wb" ) )

####### relatively small dataset, can be used at development phase #######
pickle.dump(dev_training_data, open( "dev_training_data.pkl", "wb" ) )
pickle.dump(dev_testing_data, open( "dev_testing_data.pkl", "wb" ) )

In [ ]:
#loading dataset from pickel
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id = '11CfwnSn-nN66U7YtCfCiJxSh3neZmKvk'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_training_data.pkl')  

id = '1AjVLvsK2p25eRsWWCyIqgP7fDs78UrIv'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dev_testing_data.pkl')  

id = '1lTD6bgRkmwguGAr30v-r0KBPdtnVneLb'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('testing_data.pkl')  

id = '1pCUdlZMoj99UZHtqFeza86fvVQfFmDFX'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('training_data.pkl')  


import pickle

dev_training_data = pickle.load(open("dev_training_data.pkl","rb"))
dev_testing_data = pickle.load(open("dev_testing_data.pkl","rb"))

training_data = pickle.load(open("training_data.pkl","rb"))
testing_data = pickle.load(open("testing_data.pkl","rb"))

In [ ]:
'''
# word embedding  2分 [lab2]
  创建word embedding model(表现word vectors eg: word2vec-CBOW, word2vec-Skip gram, fastText + Glove[lab2])
  input embedding of your sequence model
  [lab3,4]使用了one-hot vectors 作为inputs 给长序列模型
  【要求做的】
    1.数据的预处理（word embeddings[lab2;还要做解释说明]） NLTK Twitter dataset ( section 1 提供了) or/and 任何数据集（TED talk , Google News)
    2.创建训练模型（for word embeddings) 创建训练模型；需要说清楚超参数【lab2（dimension of embeddings; window size, learning rate,etc.】 
        note 任何word embeddings model[lab2] (word2vec-CBOW, word2vec-Skip gram, fasttext, glove)【证明】
    3.训练模型： train model

      [从lab2,4,5随便选参数]embeddings_dimension ; learning_rate epochs;
  [lab5 有模型构建]
'''

'''
先变成词向量。
1变成向量矩阵。
句子长度不同。要做padding
np.mean()求均值。

模型： lab3
RNN
0-1 loss（这个好）或 交叉熵
linear： 叫分类器。 输出俩是p还是n
batch-size； hiddenstates层数；维度

'''
